# Profil Diri
---
Nama: Muhammad Jamal Luthfi

Username: azkacrows

Sekolah  : SMK Tamansiswa Mojoagung

Email: azka.achill@gmail.com

Domisili: Jombang, Jawa Timur

---

#Import librari yang dibutuhkan

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import keras.preprocessing
from google.colab import files
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

#Download dan ekstrak dataset

In [ ]:
!wget --no-check-certificate \
 https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
 -O /tmp/rockpaperscissors.zip

local_zip = '/tmp/rockpaperscissors.zip'
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall('/tmp/')

base_dir = "/tmp/rockpaperscissors/rps-cv-images/"

#Image data generator dengan augmentasi untuk data pelatihan dan validasi

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.4
)


#Train generator data

In [ ]:
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

#Validasi data

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

#Model sekuensial

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

callback

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.96 and logs.get('val_accuracy') > 0.96:
            print("\nPelatihan dihentikan, akurasi model telah mencapai di atas 96%!")
            self.model.stop_training = True

callbacks = myCallback()

#Compile model

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#Train model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[callbacks],
    verbose=2
)

# Ploting akurasi training dan validasi

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.figure(figsize=(12, 6))
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend(loc='lower right')
plt.figure()

plt.figure(figsize=(12, 6))
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend(loc='upper right')
plt.show()

#Fungsi untuk prediksi gambar yang di upload

In [ ]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    predicted_class = class_indices[np.argmax(prediction)]
    return predicted_class


uploaded = files.upload()

for fn in uploaded.keys():
    img_path = fn
    predicted_class = predict_image(img_path)
    print(f'Kelas yang diprediksi untuk gambar yang diupload adalah: {predicted_class}')

    img = image.load_img(img_path, target_size=(150, 150))
    plt.imshow(img)
    plt.title(predicted_class)
    plt.axis('off')
    plt.show()